In [10]:
# Importing dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# Creating the path to chromedriver and setting up the browser object
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Defining the URL for scraping & visiting the website
url = "https://www.imdb.com/chart/top/"
browser.visit(url)

In [4]:
# Extract the HTML from the website and parse with BeautifulSoup
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# Search for the table which contains the information we want to scrape
table = soup.find("table", class_ = "chart")

In [6]:
# Find the titles within the table
titles = table.find_all("td", class_ = "titleColumn")

In [7]:
# Append each title name to a list
title_list = []

for title in titles:
    title_list.append(title.find("a").get_text())
    
# Confirming that all 250 titles were found
len(title_list)

250

In [8]:
# Find the ratings within the table
ratings = table.find_all("td", class_ = "ratingColumn")

In [9]:
# Append each rating to a list
rating_list = []

for i in range(0, len(ratings)):
    if i % 2 == 0:
        rating_list.append(ratings[i].find("strong").get_text())
        
# Confirming that all 250 titles were found
len(rating_list)

250

In [11]:
top_movies_df = pd.DataFrame({"Title": title_list, "Rating": rating_list})

In [12]:
top_movies_df

,Title,Rating
0,The Shawshank Redemption,9.2
1,The Godfather,9.1
2,The Godfather: Part II,9.0
3,The Dark Knight,9.0
4,12 Angry Men,8.9
...,...,...
245,The Battle of Algiers,8.0
246,The Terminator,8.0
247,Tangerines,8.0
248,A Silent Voice: The Movie,8.0
